In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
#installing required libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
#from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [3]:
#Read train set
train = pd.read_csv('train.csv')
pd.DataFrame(train)

,Image_File,Class
0,clean2416.png,1
1,ground4022.png,0
2,ground0340.png,0
3,ground2110.png,0
4,clean1581.png,1
5,clean0555.png,1
6,ground0945.png,0
7,ground0960.png,0
8,ground1373.png,0
9,ground2361.png,0


In [10]:
#Read training images, convert them into arrays, store values into *train_image* array:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('Train Images/'+train['Image_File'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 11998/11998 [03:53<00:00, 51.30it/s]


In [11]:
#one-hot encoding
y=train['Class'].values
y = to_categorical(y)

In [12]:
#Split training set into tain-set, cross-validation set as 80% and 20% respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [23]:
#CNN part
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [24]:
#Compile model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [26]:
#Validate the model using 20% cross-validation set by training 80% data
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 9598 samples, validate on 2400 samples
Epoch 1/10
9598/9598 [==============================] - 45s 5ms/step - loss: 0.2458 - accuracy: 0.8996 - val_loss: 0.0853 - val_accuracy: 0.9712
Epoch 2/10
9598/9598 [==============================] - 52s 5ms/step - loss: 0.0724 - accuracy: 0.9750 - val_loss: 0.0418 - val_accuracy: 0.9837
Epoch 3/10
9598/9598 [==============================] - 50s 5ms/step - loss: 0.0474 - accuracy: 0.9840 - val_loss: 0.0365 - val_accuracy: 0.9871
Epoch 4/10
9598/9598 [==============================] - 50s 5ms/step - loss: 0.0190 - accuracy: 0.9935 - val_loss: 0.0194 - val_accuracy: 0.9896
Epoch 5/10
9598/9598 [==============================] - 50s 5ms/step - loss: 0.0153 - accuracy: 0.9942 - val_loss: 0.0148 - val_accuracy: 0.9954
Epoch 6/10
9598/9598 [==============================] - 49s 5ms/step - loss: 0.0076 - accuracy: 0.9978 - val_loss: 0.0070 - val_accuracy: 0.9979
Epoch 7/10
9598/9598 [==============================] - 49s 5ms/step - loss: 0.012

In [28]:
#Check the test set csv
test = pd.read_csv('test.csv')
pd.DataFrame(test)

,Image_File,Class
0,lg 988 (1).png,NaN
1,lg 988 (10).png,NaN
2,lg 988 (100).png,NaN
3,lg 988 (101).png,NaN
4,lg 988 (102).png,NaN
5,lg 988 (103).png,NaN
6,lg 988 (104).png,NaN
7,lg 988 (105).png,NaN
8,lg 988 (106).png,NaN
9,lg 988 (107).png,NaN


In [29]:
#Read test images, convert the, into arrays, store values into *test_image* array:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('Test Images/'+test['Image_File'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 7534/7534 [03:38<00:00, 34.56it/s]


In [30]:
#Prediction
prediction = model.predict_classes(test)

In [31]:
#Creating a submission file with predicted outputs
sample = pd.read_csv('test.csv')
sample['Class'] = prediction
sample.to_csv('sample_submission.csv', header=True, index=False)